In [88]:
import matplotlib.pyplot as plt
import seaborn as sns
# Configs
%config InlineBackend.figure_format ='retina'
%matplotlib inline

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn import base, neural_network, neighbors, feature_extraction, datasets, linear_model, metrics, model_selection, preprocessing, svm, ensemble, decomposition, feature_extraction, utils

import keras


In [89]:
# Load Bridge helper classes
%run helpers/bridge.ipynb
%run helpers/load-data.ipynb
%run helpers/train-eval-utils.ipynb

In [91]:
# Loads & precomputes all the data, takes about 2.5min
data = load_deals()

In [85]:
# Concatenating features
X = [np.array([h.feature_array() for h in d.hands]).flatten()
     for d in data]

# Winners in a particular strain played from N
y = [d.wnrs[strains.index('NT')]['N'] for d in data]

In [87]:
m = train_eval_dds({'m': neural_network.MLPRegressor(max_iter=500), 'desc': 'Sklearn-NN'}, ret_model=True) # 4 min

Features: Globally-defined X
    Targets: Globally-defined y

        M: Sklearn-NN #  test MAE: 0.76 - tr MAE: 0.72 - T: 256.4s # 40.2|86.2|97.8


In [86]:
m = train_eval_dds({'m': neural_network.MLPRegressor(), 'desc': 'Sklearn-NN'}, ret_model=True) # 4 min

Features: Globally-defined X
    Targets: Globally-defined y



/Applications/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


        M: Sklearn-NN #  test MAE: 0.76 - tr MAE: 0.72 - T: 230.6s # 40.3|86.3|97.7


In [77]:
m = train_eval_dds({'m': ensemble.HistGradientBoostingRegressor(), 'desc': 'HistGB    '}, ret_model=True)


Features: Globally-defined X
    Targets: Globally-defined y

        M: HistGB     #  test MAE: 0.84 - tr MAE: 0.82 - T: 6.6s # 36.3|83.6|97.0


In [78]:
train_eval_dds({'m': linear_model.LinearRegression(), 'desc': 'LinearRegr'})

Features: Globally-defined X
    Targets: Globally-defined y

        M: LinearRegr #  test MAE: 0.97 - tr MAE: 0.96 - T: 0.7s # 32.3|77.6|94.8


In [52]:
train_eval_dds({'m': seq_model, 'desc': 'MLPNadamTan',
            'b_args': {'activation': 'tanh', 'optimizer': keras.optimizers.Nadam()},
            'f_args': {'batch_size': 32, 'epochs': 10, 'verbose': 0}})  # 3min on CPU

Features: Globally-defined X
    Targets: Globally-defined y



KeyboardInterrupt: 